# 1. Evolve with SUPG, steady state. 

## Exact Solution (Iliescu Ex. 4.1)
For each step choose an exact solution and compute the solution for for all the refinement levels. Make sure that there is at least one refinement level on which the standard Galerkin does not show oscillations.



In [1]:
%matplotlib inline 
import matplotlib
from dolfin import *
from __future__ import print_function
import numpy as np
import math as m
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])
# Exact Solution
c = 16.0 #*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
iliescu = c*h*(0.5+sym.atan(g)/sym.pi)

ue = sym.simplify(iliescu)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = 0 #sym.diff(ue,t) Remove time dependence for now
fe += - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')


In [2]:
def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	return L2n, H1n

def compute_extrema(u, t):
    maxval = np.amax(u.vector().get_local())
    minval = np.amin(u.vector().get_local())
    return maxval, minval

In [3]:
def evolveSUPG(nx, u_code, f_code, sigma, mu, velocity):
    degree = 2

    u_exact = Expression(u_code, degree = degree)
    
    f = Expression(f_code, degree = degree)

    mesh = UnitSquareMesh(nx,nx)

    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree)
    def boundary(x, on_boundary):
        return on_boundary
    
    bc = DirichletBC(Q, u_D, boundary)

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    u_n = Function(Q)
    u_ = Function(Q)

    # Galerkin variational problem
    F = mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx

    # SUPG Stabilization terms
    h = CellDiameter(mesh)
    r = - mu*div(grad(u)) + dot(velocity, grad(u)) + sigma*u - f
    vnorm = sqrt(dot(velocity, velocity))
    F += (h/(2.0*vnorm))*dot(velocity, grad(v))*r*dx
    
    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)
    bc.apply(A1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)

    # Step 1 Solve on Coarse Grid
    b = assemble(L)
    bc.apply(b)
    solve(A1, u_.vector(), b, 'gmres')
    
    out_file_uexact = File("SUPG/ES_steady_u_exact_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    out_file_uexact << ue
    out_file_ubar = File("SUPG/"+str(mu)+"ES_steady_u_"+str(nx)+".pvd") 
    out_file_ubar << u_
    
    L2, H1 = compute_errors(u_exact, u_, t, mesh)
    maxval, minval = compute_extrema(u_, t)
    
    print(nx,",",L2,",",H1,",",maxval,",",minval)

In [5]:
for nx in [25, 50, 100, 200, 400]:
    evolveSUPG(nx, u_code, f_code, sigma, mu, b)

25 , 0.245968068399 , 17.1895838925 , 1.73328566595 , -1.20885603562
50 , 0.133728459777 , 12.3432897754 , 1.36169871202 , -0.527724650346
100 , 0.0237896053524 , 5.54948066129 , 1.05746493627 , -0.127895091942
200 , 0.00410059233595 , 1.85798927312 , 0.994636709422 , -0.0187391834509
400 , 0.000160612398628 , 0.282522994955 , 0.991952997451 , -0.000362749212657
